In [94]:
import numpy as np
import pandas as pd
from os.path import isfile 

import brightway2 as bw2
from bw2data.parameters import ActivityParameter, ProjectParameter, DatabaseParameter, Group

from contributionAnalysis import * 

import lca_algebraic as la

In [95]:
# create a new bw2 project & load ecoinvent data in it
if "demo4contest" not in bw2.projects:
    bw2.projects.create_project("demo4contest")
    bw2.bw2setup()
else:
    print("Project already exists.")
    bw2.projects.set_current("demo4contest")
    
#ecoinvent
if 'ei38_cutoff' in bw2.databases:
    print("Database has already been imported.")
else:
    # mind that the ecoinvent file must be unzipped; then: path to the datasets subfolder
    fpei = r"C:\Users\Elias S. Azzi\Downloads\ecoinvent 3.8_cutoff_ecoSpold02\datasets"
    # the "r" makes sure that the path is read as a string - especially useful when you have spaces in your string
    ei = bw2.SingleOutputEcospold2Importer(fpei, 'ei38_cutoff')
    ei.apply_strategies()
    ei.statistics()
    ei.write_database()
bw2.databases

Project already exists.
Database has already been imported.


Databases dictionary with 3 object(s):
	biosphere3
	ei38_cutoff
	foreground

In [96]:
# develop a parametrized LCA model (here a random quick example is made, but you can do your own the way you like)
USER_DB = 'foreground'
la.resetDb(USER_DB)
la.setForeground('foreground')

#some parameters
a = la.newFloatParam('p_a', default=0.5, min=0.2, max=2, description="hello world")
b1 = la.newFloatParam('p_b', default=0.5, min=0.2, max=2, description="foo bar")
b2 = la.newFloatParam('p_be', default=0.5, min=0.2, max=2, description="foo bar")
cem = la.newFloatParam('p_cem', default=0.5, min=0.2, max=2, description="foo bar")
c = la.newFloatParam('p_c', default=0.5, min=0.2, max=2, description="foo bar")
e1 = la.newFloatParam('p_e', default=0.5, min=0.2, max=2, description="foo bar")
e2 = la.newFloatParam('p_ee', default=0.5, min=0.2, max=2, description="foo bar")
f = la.newFloatParam('p_f', default=0.5, min=0.2, max=2, description="foo bar")
d = la.newFloatParam('p_d', default=0.5, min=0.2, max=2, description="foo bar")
el = la.newFloatParam('p_el', default=0.5, min=0.2, max=2, description="foo bar")
al = la.newFloatParam('p_al', default=0.5, min=0.2, max=2, description="foo bar")

# some background inputs
alu = la.findTechAct("aluminium alloy production, AlMg3", loc="RER")
elec = la.findTechAct("market for electricity, high voltage", loc="SE")
cement = la.findTechAct("market for cement, Portland", loc="Europe without Switzerland")
transport = la.findTechAct("transport, freight, lorry, all sizes, EURO5 to generic market for transport, freight, lorry, unspecified", loc="RER")

# some foreground activities
F = la.newActivity(USER_DB, "F", 'ton*km', {transport:1})
E = la.newActivity(USER_DB, "E", 'ton*km', {transport:1})
D = la.newActivity(USER_DB, "D", 'ton*km', {transport:1})
C = la.newActivity(USER_DB, "C", 'ton*km', {
    D:d,
    elec:el,
})
B = la.newActivity(USER_DB, "B", 'ton*km', {
    F:f,
    alu:al,
})
A = la.newActivity(USER_DB, "A", 'ton*km', {
    E:e1*e2,
    cement:cem,
    C:c,
})
FU = la.newActivity(USER_DB, "FU", 'magic', {
    A:a,
    B:b1*10+b2,
})

Db foreground was here. Reseting it
[ParamRegistry] Param p_a was already defined in '<project>' : overriding.
[ParamRegistry] Param p_b was already defined in '<project>' : overriding.
[ParamRegistry] Param p_be was already defined in '<project>' : overriding.
[ParamRegistry] Param p_cem was already defined in '<project>' : overriding.
[ParamRegistry] Param p_c was already defined in '<project>' : overriding.
[ParamRegistry] Param p_e was already defined in '<project>' : overriding.
[ParamRegistry] Param p_ee was already defined in '<project>' : overriding.
[ParamRegistry] Param p_f was already defined in '<project>' : overriding.
[ParamRegistry] Param p_d was already defined in '<project>' : overriding.
[ParamRegistry] Param p_el was already defined in '<project>' : overriding.
[ParamRegistry] Param p_al was already defined in '<project>' : overriding.


The code above craetes the following LCA model (image generated from graph explorer in Activity-browser):

![graph view](demo4contest.png)

In [97]:
# define some contribution groups that make sense for the analysis
A['group']="Step Alpha"
A.save()
D['group']="All transport"
D.save()
E['group']="All transport"
E.save()
F['group']="All transport"
F.save()
C['group']="Step Gamma"
C.save()
B['group']="Step Beta"
B.save()

The grouping tags applied in the cell above correspond to grouping the emissions as shown in the (rudimentary) image below: 

![graph view](demo4contest_groupped.png)

In [98]:
# setup your LCA calculation
fus = [{FU:1},] # a list of fu dict
fus_prettyNames = ['FU']
IPCC = [method for method in bw2.methods if "IPCC 2013" in str(method) 
        and "GWP 100" in str(method) 
        and "LT" not in str(method)
        and "V1" not in str(method)] # a list of tuples (tuple refering to a bw2 impact assessment metric)

label = 'group'
default_tag = 'other' # in case an activity is uncategorized
bio2tech = True
parent4other = True
contribution_oder = ['Step Alpha', 'Step Beta', 'Step Gamma', 'All transport', 'other']

prefix = [] # prefix of switch parameters, defined manually in project, if any (Not used here in demo)
alternatives = {} # parameter data for switch parameters, if any (Not used here in demo)

afp = r'C:\github\bw2-widgets\html_export' # where to export
widget_name="widget_waterfall_demo4contest" # name of widget

#setupProject4Widget("demo4contest", USER_DB)

In [99]:
# generate algebraic expressions & parameter data
all_graphs = runAlgebraicGraphTraversal(fus = fus, # a list of fu dict 
                           methods =IPCC, # a list
                           label = label, # a string
                           default_tag = default_tag, # a string
                           fg_databases = [USER_DB], # a list of 1 foreground db
                           bio2tech = bio2tech, # usually, True
                           parent4other = parent4other) # usually, True
print("The contributions present in the graphs are: ")
contribExisting = []
for fu, graphs in all_graphs.items():
    contribExisting += list(graphs['agg_graph'].keys())
print(set(contribExisting))

# re-order contributions according to contribution_order
for fu, graphs in all_graphs.items():
    all_graphs[fu]['agg_graph'] = reorderContributions(all_graphs[fu]['agg_graph'], contribution_oder)
    print("Contributions re-ordered for: %s" % (fu))
    

The contributions present in the graphs are: 
{'Step Gamma', 'Step Beta', 'All transport', 'other', 'Step Alpha'}
Contributions re-ordered for: 'FU' (magic, GLO, None)


In [100]:
# saving the parameter data in an Excel file... to be made prettier
all_Symbols = []
for fu, graphs in all_graphs.items():
    all_Symbols += getAllSymbols(all_graphs[fu]['agg_graph']) # concat of lists += getAllSymbols returning a list
all_Symbols = list(set(all_Symbols)) # all parameters, even if several fus analysed
df = getParamsFromSymbols(all_Symbols)
jstr_paraminit = genJSvariables(df) # placed here, before df = getSwitchParams
df = getSwitchParams(df, prefix, alternatives) # remove from df the switch parameters, replace them by a single generic term
df = df.drop(['group', 'label'], axis=1)
jsstr_switchparam = str(alternatives)
df.to_excel(afp+'\\'+widget_name+'_params.xlsx')
print("Data of parameters was written to the Excel file \n")
print(afp+'\\'+widget_name+'_params.xlsx \n')
print("Proceed with: dupplicating this Excel file, and rename it with the suffix '_prettified'. \nThe prettified Excel file is never overwritten or edited by the scripts here. If it exists, it will be loaded in the next step. This is to avoid overwriting by mistake data entered manually in Excel.\n")
print("In the prettified Excel, you must not edit the column 'name' but you can edit the field `min`, `max`, `Description`. You can also add the columns `prettyName` and `unit`")

Data of parameters was written to the Excel file 

C:\github\bw2-widgets\html_export\widget_waterfall_demo4contest_params.xlsx 

Proceed with: dupplicating this Excel file, and rename it with the suffix '_prettified'. 
The prettified Excel file is never overwritten or edited by the scripts here. If it exists, it will be loaded in the next step. This is to avoid overwriting by mistake data entered manually in Excel.

In the prettified Excel, you must not edit the column 'name' but you can edit the field `min`, `max`, `Description`. You can also add the columns `prettyName` and `unit`


In [101]:
# manually edit the excel file to refine parameter data, give names and units that will be displayed

In [103]:
# finally export the js widget
if(isfile(afp+'\\'+widget_name+'_params_prettified.xlsx')):
    # if a prettified version exist, we take that one
    df2 = pd.read_excel(afp+'\\'+widget_name+'_params_prettified.xlsx', index_col=0).fillna('')
jsstr_param = str(df2.to_dict())

jstr_algeabraicfuncs = ''
for i, (fu, graphs) in enumerate(all_graphs.items()):
    # hack: remove [ ] around each contribution if 1 element only (1 impact category)
    for key, values in graphs['agg_graph'].items():
        if(isinstance(values, list)):
            if(len(values)>1):
                    print("Warning: multiple impact categories calculated, but only first one kept for plugin as not yet supported graphs")
            graphs['agg_graph'][key] = values[0]    
    # add group-series pretty name
    graphs['agg_graph']['group']=fus_prettyNames[i]
    jstr_algeabraicfuncs += str(graphs['agg_graph'])+', '
    
write_widget(afp+'\\'+widget_name+'.js', jsstr_param, jsstr_switchparam, jstr_paraminit, jstr_algeabraicfuncs)
print("Load this file in an html demo page")

File created in: C:\github\bw2-widgets\html_export\widget_waterfall_demo4contest.js
Load this file in an html demo page


Note: be mindful of the end-user license agreements of the databases used (e.g. ecoinvent) before sharing such widgets externally..